# Watershed Distance Transform for 2D Data
---
Implementation of papers:

[Deep Watershed Transform for Instance Segmentation](http://openaccess.thecvf.com/content_cvpr_2017/papers/Bai_Deep_Watershed_Transform_CVPR_2017_paper.pdf)

[Learn to segment single cells with deep distance estimator and deep cell detector](https://arxiv.org/abs/1803.10829)

In [1]:
import os
import errno

import numpy as np

import deepcell

/usr/local/lib/python3.5/dist-packages/deepcell/utils/__init__.py:49: UserWarning: To use `compute_overlap`, the C extensions must be built using `python setup.py build_ext --inplace`
  warnings.warn('To use `compute_overlap`, the C extensions must be built '


In [2]:
from tensorflow.keras.optimizers import SGD
from deepcell.utils.train_utils import rate_scheduler

n_epoch = 10  # Number of training epochs
test_size = .10  # % of data saved as test
norm_method = 'std'  # data normalization
receptive_field = 61  # should be adjusted for the scale of the data

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

lr_sched = rate_scheduler(lr=0.01, decay=0.99)

# Sample mode settings
batch_size = 64  # number of images per batch (should be 2 ^ n)
win = (receptive_field - 1) // 2  # sample window size
balance_classes = True  # sample each class equally
max_class_samples = 1e6  # max number of samples per class

# Transformation settings
transform = 'watershed'
distance_bins = 4  # number of distance "classes"
erosion_width = 0  # erode edges

In [3]:
base_name = 'Point1_12_18_3X_interior_border_border'
npz_name = '/data/npz_data/' + base_name + ".npz"
MODEL_DIR = '/data/models/' + '20190505_watershed_retrain'

if not os.path.isdir(MODEL_DIR):
    os.makedirs(MODEL_DIR)

LOG_DIR = '/data/logs'


fgbg_model_name = base_name + "_fgbg"
sample_model_name = base_name + "_watershed"

### Load the Training Data

In [4]:
CHANNEL_AXIS = 3
training_data = np.load(npz_name)

X, y = training_data["X"], training_data["y"]
print("X.shape: {} & y.shape: {}".format(X.shape, y.shape))

X.shape: (3, 1024, 1024, 3) & y.shape: (3, 1024, 1024, 1)


### First, create a foreground/background separation model

#### Instantiate the fgbg model

In [5]:
from deepcell import model_zoo

fgbg_model = model_zoo.bn_feature_net_2D(
    receptive_field=receptive_field,
    n_channels=X.shape[CHANNEL_AXIS],
    n_features=2)

#### Train the fgbg model

In [13]:
from deepcell.training import train_model_sample

fgbg_model = train_model_sample(
    model=fgbg_model,
    dataset=npz_name,
    model_name=fgbg_model_name,
    test_size=test_size,
    optimizer=optimizer,
    window_size=(win, win),
    batch_size=128,
    transform='fgbg',
    n_epoch=1,
    balance_classes=balance_classes,
    max_class_samples=100000,
    model_dir=MODEL_DIR,
    log_dir=LOG_DIR,
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False,
    zoom_range=(0.8, 1.2),
    val_monitor=False,
    save_period=1)

using all data as training data
X_train shape: (3, 1024, 1024, 3)
y_train shape: (3, 1024, 1024, 1)
Output Shape: (None, 2)
Number of Classes: 2
Training on 1 GPUs
running model without validation checks
Epoch 1/1
1561/1562 [============================>.] - ETA: 0s - loss: 0.2308 - acc: 0.9056
Epoch 00001: saving model to /data/models/20190505_watershed_retrain/Point1_12_18_3X_interior_border_border_fgbg_epoch_01.h5
1562/1562 [==============================] - 257s 165ms/step - loss: 0.2308 - acc: 0.9057


### Next, Create a model for the watershed energy transform

#### Instantiate the distance transform model

In [9]:
from deepcell import model_zoo
from deepcell.training import train_model_sample

watershed_model = model_zoo.bn_feature_net_2D(
    receptive_field=receptive_field,
    n_channels=X.shape[CHANNEL_AXIS],
    n_features=distance_bins)

#### Train the model

In [10]:
watershed_model = train_model_sample(
    model=watershed_model,
    dataset=npz_name, 
    model_name=sample_model_name,
    test_size=test_size,
    optimizer=optimizer,
    batch_size=batch_size,
    n_epoch=40,
    window_size=(win, win),
    transform=transform,
    distance_bins=distance_bins,
    erosion_width=erosion_width,
    balance_classes=balance_classes,
    max_class_samples=max_class_samples,
    model_dir=MODEL_DIR,
    log_dir=LOG_DIR,
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False,
    zoom_range=(0.8, 1.2),
    val_monitor=False,
    save_period=10)

using all data as training data
X_train shape: (3, 1024, 1024, 3)
y_train shape: (3, 1024, 1024, 1)
Output Shape: (None, 4)
Number of Classes: 4
Training on 1 GPUs
running model without validation checks
Epoch 1/40
6382/6382 [==============================] - 550s 86ms/step - loss: 0.7619 - acc: 0.6272
Epoch 2/40
6382/6382 [==============================] - 535s 84ms/step - loss: 0.7248 - acc: 0.6460
Epoch 3/40
6382/6382 [==============================] - 532s 83ms/step - loss: 0.7128 - acc: 0.6522
Epoch 4/40
6382/6382 [==============================] - 528s 83ms/step - loss: 0.7044 - acc: 0.6560
Epoch 5/40
6382/6382 [==============================] - 528s 83ms/step - loss: 0.6967 - acc: 0.6601
Epoch 6/40
6382/6382 [==============================] - 524s 82ms/step - loss: 0.6925 - acc: 0.6621
Epoch 7/40
6382/6382 [==============================] - 521s 82ms/step - loss: 0.6865 - acc: 0.6661
Epoch 8/40
6382/6382 [==============================] - 518s 81ms/step - loss: 0.6813 - acc: 0.6